In [ ]:
# Basic setup
%run ./config.ipynb

In [ ]:
from cortex import Message, Cortex
from langdetect import detect_langs


def detect(params):
    msg = Message(params)
    cortex= Cortex.client(msg.apiEndpoint, token = msg.token)
    text = msg.payload.get('text')

    results = detect_langs(text)

    others = []
    for lang in results[1:]:
        others.append({'lang': lang.lang, 'score': lang.prob})

    top_lang = results[0]

    return cortex.message(
        {'text': text, 'lang': top_lang.lang, 'score': top_lang.prob, 'other_langs': others}).to_params()

In [ ]:
import sys
import argparse
from pprint import pprint


def test_detect(args):
    text = u'¡Bienvenido a CognitiveScale, que tengas un buen día!'
    print('Detecting language in: {}'.format(text))
    m = Message.with_payload({'text': text})
    result = detect(m.to_params())
    pprint(result['payload'])

test_detect(None)

In [ ]:
import sys
from cortex import Cortex

_shim = """
def main(params):
    return %s(params)
"""


def load_source(path, fn):
    with open(path) as f:
        source = f.read()
    return '{}\n\n{}'.format(source, _shim % fn)


def build_detect_action(builder):
    action = builder.action('c12e/lang-detect').from_source(
        load_source('lang_detect/detect.py', 'detect')).with_requirements(['langdetect']).daemon().build()

    return action


def build_detect_skill(builder, action):
    b = builder.skill('c12e/lang-detect-skill').title('Language Detection').description(
        'Detects the language used in a provided text.')

    b.input('text_input').parameter('text', 'string', None, 'Text').all_routing(
        action, 'text_with_language').build()

    b.output('text_with_language').parameter('text', 'string', None, 'Text').parameter(
        'lang', 'string', None, 'Language').parameter('score', 'number', 'float', 'Score').parameter(
        'other_langs', 'array', None, 'Other Languages').build()

    return b.build()


def build_detect():
    cortex = Cortex.client()
    builder = cortex.builder()

    action = build_detect_action(builder)
    print('Deployed action {} at version v{}'.format(action.name, action.version))
    print(action.get_deployment_status())

    skill = build_detect_skill(builder, action)
    print('Deployed skill {} at version v{}'.format(skill.name, skill.version))

build_detect()
